In [21]:
!pip install crewai langchain-community langchain-openai duckduckgo-search wikipedia

In [48]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['EXCHANGERATE_API_KEY'] = userdata.get('EXCHANGERATE_API_KEY')

In [49]:
import requests
from datetime import datetime, timedelta
import json
from crewai import Agent, Task, Crew
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_openai import ChatOpenAI

# Creating Custom Tool

In [50]:
@tool
def wikipedia_search(query: str) -> str:
    """Search Wikipedia for detailed information about places, people, and concepts."""
    return wikipedia.run(query)

In [52]:
class CurrencyConverter:
    def __init__(self):
        self.cache = {}
        self.last_update = None

    def get_rates(self, base_currency='USD'):
        """Fetch current exchange rates from API with caching"""
        if self.last_update and datetime.now() - self.last_update < timedelta(hours=1):
            return self.cache

        try:
            api_key = os.getenv("EXCHANGERATE_API_KEY")
            response = requests.get(f'https://v6.exchangerate-api.com/v6/{api_key}/latest/{base_currency}')
            data = response.json()

            if data['result'] == 'success':
                self.cache = data['conversion_rates']
                self.last_update = datetime.now()
                return self.cache
            else:
                print(f"API Error: {data['error-type']}")
                return None
        except Exception as e:
            print(f"Currency API Error: {str(e)}")
            return None

In [53]:
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> float:
    """Convert between currencies using real-time exchange rates from 160+ currencies"""
    converter = CurrencyConverter()
    rates = converter.get_rates()

    if not rates:
        return "Currency service unavailable. Please try again later."

    from_currency = from_currency.upper()
    to_currency = to_currency.upper()

    if from_currency not in rates or to_currency not in rates:
        return f"Invalid currency code. Supported codes: {', '.join(rates.keys())}"

    # Convert via USD base rates
    usd_amount = amount / rates[from_currency]
    converted_amount = usd_amount * rates[to_currency]

    return round(converted_amount, 2)



In [56]:
search_tool = DuckDuckGoSearchRun().as_tool()
wikipedia = WikipediaAPIWrapper()

# Creating Agents

In [93]:
destination_researcher = Agent(
    role='Senior Travel Researcher',
    goal='Find ideal travel destinations based on user preferences',
    backstory="""A seasoned travel expert with deep knowledge of global destinations and
    cultural experiences. Known for matching locations to traveler personalities.""",
    tools=[search_tool, wikipedia_search],
    verbose=True,
    allow_delegation=False,
    llm=ChatOpenAI(
        model_name='gpt-4o',
        temperature=0.7,
        max_tokens=2000
    )
)

In [94]:
itinerary_planner = Agent(
    role='Master Itinerary Designer',
    goal='Create detailed daily itineraries with activities and logistics',
    backstory="""A professional travel planner with decade of experience crafting
    seamless vacation experiences. Expert in transportation logistics and local
    activity coordination.""",
    tools=[search_tool],
    verbose=True,
    allow_delegation=False,
    llm=ChatOpenAI(model_name='gpt-4o', temperature=0.5)
)

In [95]:
budget_advisor = Agent(
    role='Financial Travel Consultant',
    goal='Optimize travel plans to meet budget constraints',
    backstory="""A former tour company CFO who specializes in budget optimization for
    travel experiences. Expert in cost-benefit analysis of travel options.""",
    tools=[currency_converter, search_tool],
    verbose=True,
    allow_delegation=False,
    llm=ChatOpenAI(model_name='gpt-4o', temperature=0.3)
)

In [96]:
travel_coordinator = Agent(
    role='Travel Logistics Coordinator',
    goal='Handle bookings and practical travel arrangements',
    backstory="""A detail-oriented travel operations manager with global network of
    transportation and accommodation partners. Known for flawless execution.""",
    tools=[search_tool],
    verbose=True,
    allow_delegation=False,
    llm=ChatOpenAI(model_name='gpt-4o', temperature=0.2)
)

# Creating Tasks

In [97]:
research_task = Task(
    description="""Research potential destinations for a {traveler_type} traveler
    interested in {interests}. Consider safety, seasonal weather {season}, and
    cultural relevance.""",
    expected_output="Bulleted list of 5-7 potential destinations with key highlights",
    agent=destination_researcher,
    output_file="destination_report.md"
)

In [98]:
planning_task = Task(
    description="""Create a {days}-day itinerary based on the researcher's destinations including:
    - Daily themes
    - Morning/Afternoon/Evening activities
    - Dining recommendations
    - Transportation between locations""",
    expected_output="Detailed day-by-day schedule in Markdown format",
    agent=itinerary_planner,
    context=[research_task],  # Gets destinations from research_task output
    output_file="travel_itinerary.md"
)

In [99]:
budgeting_task = Task(
    description="""Calculate estimated costs for the planned itinerary with breakdown of:
    - Accommodation
    - Transportation
    - Activities
    - Food
    Convert all costs to {currency} currency""",
    expected_output="Detailed budget table with category costs and total estimate",
    agent=budget_advisor,
    context=[planning_task],  # Automatically gets itinerary details
    output_file="budget_report.md"
)

In [100]:
booking_task = Task(
    description="""Generate practical booking information for the travel plan:
    - Recommended booking platforms
    - Visa requirements
    - Vaccination needs
    - Local emergency contacts
    - Transportation options
    - Accommodation suggestions""",
    expected_output="Bullet-point list of booking resources and travel requirements",
    agent=travel_coordinator,
    context=[planning_task],
    output_file="booking_guide.md"
)

# Creating the Crew

In [ ]:
travel_crew = Crew(
    agents=[destination_researcher, itinerary_planner, budget_advisor, travel_coordinator],
    tasks=[research_task, planning_task, budgeting_task, booking_task],
    verbose = True,
    process='sequential'
)

# Running the Crew

In [102]:
inputs = {
    'traveler_type': 'adventure-seeking couple',
    'interests': ['outdoor activities', 'local cuisine', 'historical sites'],
    'season': 'summer',
    'days': 10,
    'currency': 'EUR'
}

result = travel_crew.kickoff(inputs=inputs)




# inputs = {
#     'traveler_type': 'family with young children (ages 5-8)',
#     'interests': ['theme parks', 'child-friendly beaches', 'interactive museums'],
#     'season': 'spring',
#     'days': 7,
#     'currency': 'USD'
# }




# inputs = {
#     'traveler_type': 'solo budget backpacker',
#     'interests': ['hiking trails', 'local street food', 'hostel experiences'],
#     'season': 'fall',
#     'days': 14,
#     'currency': 'EUR'
# }



# inputs = {
#     'traveler_type': 'retired couple interested in cultural immersion',
#     'interests': ['UNESCO heritage sites', 'traditional performances', 'art galleries'],
#     'season': 'autumn',
#     'days': 10,
#     'currency': 'GBP'
# }



# Agent: Senior Travel Researcher
## Task: Research potential destinations for a adventure-seeking couple traveler 
    interested in ['outdoor activities', 'local cuisine', 'historical sites']. Consider safety, seasonal weather summer, and 
    cultural relevance.


# Agent: Senior Travel Researcher
## Using tool: duckduckgo_search
## Tool Input: 
"{\"query\": \"best adventure destinations with outdoor activities, local cuisine, historical sites, safe for summer 2023\"}"
## Tool Output: 
Patagonia is one of the best adventure destinations in the world, as it is completely centered around outdoor adventures. ... the country in the UK, is a great place to travel for adventure lovers and for people who enjoy outdoor activities. Three national parks cover one-fifth of the total Welsh land. ... but best carried out in the Spring ... These places provide heart-pounding adventures, whether you are plunging into the ocean depths, hiking through vast wilderness, or climbing tall hills. These 1

In [103]:
print("\n\n=== FINAL TRAVEL PLAN ===")
print(result)



=== FINAL TRAVEL PLAN ===
- **Recommended Booking Platforms:**
  - Kayak
  - Booking.com
  - Expedia
  - Skyscanner
  - Omio

- **Visa Requirements:**
  - **Chile/Argentina:** Visa-free for up to 90 days for many nationalities including US, Canada, EU.
  - **Japan:** Visa-free for up to 90 days for US, Canada, EU citizens.
  - **Italy:** Schengen visa required for non-EU citizens.
  - **South Africa:** Visa-free for up to 90 days for US, Canada, EU citizens.
  - **Peru:** Visa-free for up to 183 days for many nationalities.
  - **New Zealand:** Visa-free for up to 90 days for US, Canada, EU citizens.
  - **Canada:** Visa-free for up to 6 months for US citizens.

- **Vaccination Needs:**
  - **General:** Routine vaccines (MMR, Diphtheria-Tetanus-Pertussis, Varicella, Polio, Flu).
  - **Yellow Fever:** Required for South America and Africa if traveling from a risk area.
  - **COVID-19:** Check current entry requirements for each country.

- **Local Emergency Contacts:**
  - **Chile/Arg

In [106]:
from IPython.display import Markdown
Markdown(filename='destination_report.md')

- **Patagonia, Chile/Argentina**
  - Key Highlights: Hiking through Torres del Paine National Park, exploring the Perito Moreno Glacier, and experiencing local Patagonian cuisine.
  - Considerations: Best visited in the Southern Hemisphere's summer (December to February) for milder weather.

- **Kyoto, Japan**
  - Key Highlights: Visiting historical sites like Kinkaku-ji and Fushimi Inari-taisha, enjoying traditional kaiseki cuisine, and participating in outdoor activities such as hiking in the Arashiyama Bamboo Forest.
  - Considerations: Safe destination with rich cultural experiences, ideal in the summer months.

- **Tuscany, Italy**
  - Key Highlights: Exploring historical towns like Florence and Siena, indulging in Tuscan cuisine, and enjoying outdoor activities such as cycling through the countryside and wine tasting tours.
  - Considerations: Warm summer weather and a safe, culturally immersive experience.

- **Cape Town, South Africa**
  - Key Highlights: Hiking Table Mountain, exploring Robben Island and the Cape of Good Hope, and experiencing local South African cuisine.
  - Considerations: Offers a combination of adventure, history, and culinary delights. Best visited during the Southern Hemisphere's winter (May to August) for cooler weather.

- **Cusco and the Sacred Valley, Peru**
  - Key Highlights: Visiting Machu Picchu, exploring the Incan ruins, and enjoying Peruvian cuisine with local specialties such as ceviche and alpaca.
  - Considerations: Offers rich historical sites and outdoor activities. Best visited in the dry season from May to September.

- **New Zealand (South Island)**
  - Key Highlights: Engaging in outdoor activities like bungee jumping in Queenstown, hiking in Fiordland National Park, and experiencing Maori culture.
  - Considerations: Safe and adventurous destination with temperate summer weather.

- **British Columbia, Canada**
  - Key Highlights: Exploring the Canadian Rockies, engaging in kayaking and hiking, and tasting local Canadian and Indigenous cuisines.
  - Considerations: Safe destination with diverse outdoor activities and cultural experiences, optimal in summer for pleasant weather.

In [107]:
Markdown(filename='travel_itinerary.md')

# 10-Day Travel Itinerary

## Day 1-2: Patagonia, Chile/Argentina
**Theme:** Adventure and Nature

**Day 1:**
- **Morning:** Arrive in Puerto Natales, Chile. Take a bus to Torres del Paine National Park.
- **Afternoon:** Begin with a hike to the base of the Torres del Paine towers.
- **Evening:** Dinner at a local restaurant in Puerto Natales featuring Patagonian lamb.

**Day 2:**
- **Morning:** Explore the Grey Glacier via a boat tour.
- **Afternoon:** Visit the Salto Grande waterfall.
- **Evening:** Enjoy a traditional Patagonian barbecue dinner.

**Transportation:** Bus from Puerto Natales to Torres del Paine. Local shuttles within the park.

## Day 3-4: Kyoto, Japan
**Theme:** Cultural Exploration

**Day 3:**
- **Morning:** Visit Kinkaku-ji (Golden Pavilion).
- **Afternoon:** Walk through the Arashiyama Bamboo Forest.
- **Evening:** Dinner at a Kaiseki restaurant.

**Day 4:**
- **Morning:** Explore Fushimi Inari-taisha Shrine.
- **Afternoon:** Participate in a tea ceremony.
- **Evening:** Dine at an Izakaya in Gion district.

**Transportation:** Use of JR Pass for Shinkansen and local trains.

## Day 5-6: Tuscany, Italy
**Theme:** History and Gastronomy

**Day 5:**
- **Morning:** Arrive in Florence. Visit the Uffizi Gallery.
- **Afternoon:** Explore the Florence Cathedral.
- **Evening:** Dinner at a trattoria serving Tuscan specialties.

**Day 6:**
- **Morning:** Day trip to Siena. Visit the Piazza del Campo.
- **Afternoon:** Wine tasting tour in Chianti.
- **Evening:** Return to Florence for a pasta-making class.

**Transportation:** Train from Florence to Siena. Local buses and walking.

## Day 7-8: Cape Town, South Africa
**Theme:** Adventure and History

**Day 7:**
- **Morning:** Hike up Table Mountain.
- **Afternoon:** Visit Robben Island.
- **Evening:** Dinner at Mama Africa with live music.

**Day 8:**
- **Morning:** Tour of the Cape of Good Hope.
- **Afternoon:** Explore the Kirstenbosch Botanical Gardens.
- **Evening:** Enjoy seafood at the V&A Waterfront.

**Transportation:** Use of local taxis and guided tours.

## Day 9: Cusco and the Sacred Valley, Peru
**Theme:** Incan Heritage

- **Morning:** Arrive in Cusco. Acclimate with a light city tour.
- **Afternoon:** Visit Sacsayhuamán.
- **Evening:** Dinner featuring Peruvian ceviche and alpaca dishes.

**Transportation:** Local taxis and guided tours.

## Day 10: New Zealand (South Island)
**Theme:** Adventure and Scenery

- **Morning:** Arrive in Queenstown. Engage in bungee jumping.
- **Afternoon:** Explore Fiordland National Park with a hike.
- **Evening:** Dinner featuring local New Zealand lamb.

**Transportation:** Rental car for flexibility.

## Additional Days for British Columbia, Canada
**Theme:** Nature and Culture

- **Morning:** Explore Vancouver’s Stanley Park.
- **Afternoon:** Kayak in Deep Cove.
- **Evening:** Dine on Indigenous cuisine in Vancouver.

**Transportation:** Use of public transit and rental car for trips outside the city.

---

This itinerary provides a diverse range of cultural, historical, and adventurous activities across each destination, with thoughtful dining options and efficient transportation logistics to ensure a seamless travel experience.

In [109]:
Markdown(filename='budget_report.md')

The estimated costs for the planned itinerary in EUR are as follows:

- **Puerto Natales, Chile:**
  - Accommodation: 56.23 EUR per night
  - Transportation: 63.01 EUR per day
  - Activities: 38.78 EUR per day
  - Food: 7.13 EUR per meal

- **Kyoto, Japan:**
  - Accommodation: 93.06 EUR per night
  - Transportation: 1.55 EUR per ride
  - Activities: 48.47 EUR per day
  - Food: 4.36 EUR per meal

This budget table provides a detailed breakdown of costs for accommodation, transportation, activities, and food in each destination. Further research is needed to complete the budget for the remaining destinations in the itinerary.
```

In [110]:
Markdown(filename='booking_guide.md')

- **Recommended Booking Platforms:**
  - Kayak
  - Booking.com
  - Expedia
  - Skyscanner
  - Omio

- **Visa Requirements:**
  - **Chile/Argentina:** Visa-free for up to 90 days for many nationalities including US, Canada, EU.
  - **Japan:** Visa-free for up to 90 days for US, Canada, EU citizens.
  - **Italy:** Schengen visa required for non-EU citizens.
  - **South Africa:** Visa-free for up to 90 days for US, Canada, EU citizens.
  - **Peru:** Visa-free for up to 183 days for many nationalities.
  - **New Zealand:** Visa-free for up to 90 days for US, Canada, EU citizens.
  - **Canada:** Visa-free for up to 6 months for US citizens.

- **Vaccination Needs:**
  - **General:** Routine vaccines (MMR, Diphtheria-Tetanus-Pertussis, Varicella, Polio, Flu).
  - **Yellow Fever:** Required for South America and Africa if traveling from a risk area.
  - **COVID-19:** Check current entry requirements for each country.

- **Local Emergency Contacts:**
  - **Chile/Argentina:** Police: 133, Ambulance: 131
  - **Japan:** Police: 110, Ambulance/Fire: 119
  - **Italy:** Police: 112, Ambulance: 118
  - **South Africa:** Police: 10111, Ambulance: 10177
  - **Peru:** Police: 105, Ambulance: 106
  - **New Zealand:** Emergency: 111
  - **Canada:** Emergency: 911

- **Transportation Options:**
  - **Patagonia:** Buses and local shuttles.
  - **Kyoto:** JR Pass for trains, local buses.
  - **Tuscany:** Trains and local buses.
  - **Cape Town:** Local taxis and guided tours.
  - **Cusco:** Local taxis and guided tours.
  - **New Zealand:** Rental car.
  - **British Columbia:** Public transit and rental car.

- **Accommodation Suggestions:**
  - **Patagonia:** EcoCamp Patagonia, Hotel Las Torres.
  - **Kyoto:** The Ritz-Carlton, Kyoto; Hotel Granvia Kyoto.
  - **Tuscany:** Renaissance Tuscany Il Ciocco Resort & Spa.
  - **Cape Town:** One&Only Cape Town, The Silo Hotel.
  - **Cusco:** Belmond Hotel Monasterio, JW Marriott El Convento Cusco.
  - **New Zealand:** Eichardt's Private Hotel, Matakauri Lodge.
  - **British Columbia:** Fairmont Pacific Rim, The Douglas Vancouver.